In [1]:
load('Imports/Utility.sage', 'Imports/Point.sage', 'Imports/Line.sage', 'Imports/TritangentPlane.sage', 'Imports/Group.sage', 'Imports/Cubic.sage')
import multiprocessing as mp
import pickle

In [2]:
(sqrt(2)-I).minpoly('xx')

xx^4 - 2*xx^2 + 9

In [3]:
var('xx')
Q.<i> = NumberField(xx^ 2+ 1)

In [4]:
#i = ((rad2_i)^3+rad2_i)/(-6)

In [5]:
P.<x,y,z,t,b,c,d,e,f,l,m> = PolynomialRing(Q)
cubic_new = e*f*(2*x^2*y-2*x*y^2+x*z^2-x*z*t-y*t^2+y*z*t)+b*c*(x-t)*(x*z+y*t)+c*c*(z+t)*(y*t-x*z)+d*c*(y-z)*(x*z+y*t)+(e+f)*c*(x-y)*(y*t-x*z)
sing_cubics = (-1) * (-c + f) * (-c + e) * c * (c + f) * (c + e) * (-e + f)^2 * (-c*d + c*f + e*f) * (-c*d + c*e + e*f) * (-c^2 - c*d + e*f) * (b*c - c*f + e*f) * (b*c - c*e + e*f) * (b*c - c*d + 2*e*f) * (b*c - c^2 + e*f) * (b*c^2 + c^2*d + b*c*f - 2*c^2*f - c*d*f + 2*e*f^2) * (b*c^2 + c^2*d + b*c*e - 2*c^2*e - c*d*e + 2*e^2*f) * (-b*c^3 - 2*b*c^2*d + c^3*d + b*c^2*e + c^2*d*e + b*c^2*f + c^2*d*f + 3*b*c*e*f - 4*c^2*e*f - 3*c*d*e*f + 4*e^2*f^2)
line = Line([y, z])
general_cubic = Cubic(cubic_new, line, sing_cubics)
SE1 = general_cubic.subs({b:-(c*c+e*f)/c})

In [6]:
%time adm_SE1 = SE1.find_admissible_projectivities()
len(adm_SE1)

CPU times: user 5.49 s, sys: 216 ms, total: 5.71 s
Wall time: 13.5 s


576

In [7]:
%time simm_SE1 = SE1.find_simmetries(adm_SE1)
len(simm_SE1)

CPU times: user 3.55 s, sys: 107 ms, total: 3.65 s
Wall time: 18.1 s


2

In [8]:
lines_perms = []
for simm in simm_SE1:
    lines_perms.append(Permutation(SE1.apply_proj_to_lines(simm)).to_permutation_group_element())
lines_perms_group = PermutationGroup(lines_perms)
lines_perms_group.order(), lines_perms_group.structure_description()

(2, 'C2')

In [9]:
adm_perms = [from_labels_to_perm(label) for label in SE1.find_admissible_permutations()]
M2_perm = lines_perms_group.gens_small()[0]

In [10]:
adm_perms_group = PermutationGroup(adm_perms)

In [11]:
# find permutation of order 4 whose square is M2_perm
possible_perms = []
for perm in adm_perms_group:
    if perm^2 == M2_perm and perm.order() == 4:
        possible_perms.append(perm)
len(possible_perms)

12

In [12]:
possible_L_sets = [get_permuted_L_set(perm) for perm in possible_perms]
L_set_base = ['E1', 'G4', 'E2', 'G3', 'E3']

In [13]:
possible_projs = [find_projectivity(SE1.get_L_set_in_plucker(L_set_base), SE1.get_L_set_in_plucker(L_set)) for L_set in possible_L_sets]
M2 = [mat for mat in simm_SE1 if mat[0][0]!=1][0]
identity_list = [0 for _ in range(16)]
for i in {0,5,10,15}:
    identity_list[i]=1
possible_projs_squared = [proj^2 for proj in possible_projs]    

In [14]:
def find_conditions_for_subfamilies(cubic, projectivities):
    mon = ((x+y+z+t)^3).monomials()
    ideals = []
    for M in projectivities:
        print(projectivities.index(M))
        sost = change_coord(M)
        new_cubic = remove_sing_factors(cubic.eqn.subs(sost), cubic.sing_locus)    
        conds_cubic = list(set(matrix([[new_cubic.coefficient(mn) for mn in mon], [cubic.eqn.coefficient(mn) for mn in mon]]).minors(2)))
        conds_cubic = [remove_sing_factors(el, cubic.sing_locus) for el in conds_cubic if el !=0]
        #conds_square = list(set(matrix([(M^2).list(), M2.list()]).minors(2)))
        #conds_square = [remove_sing_factors(el, SE1.sing_locus) for el in conds_square if el !=0]
        conds = list(set(conds_cubic))
        ideale = cubic.P.ideal(conds)
        new_ideals = get_valid_ideals(ideale, cubic)
        for ideal in new_ideals:         
            prim_deco2 = (ideale + ideal).radical().primary_decomposition()
            for ideal2 in prim_deco2:
                if is_ideal_valid(cubic, ideal2):
                    ideals.append(ideal2)                        
    return list(set(ideals))


def get_valid_ideals(ideale, cubic):
    valid_ideals = []
    vrs = [c,d,e,f]
    for vr in vrs:
        valid_ideals_var = []
        del_ideale = ideale.elimination_ideal([vr])
        prim_deco_del = del_ideale.radical().primary_decomposition()
        for ideal in prim_deco_del:
            if is_ideal_valid(cubic, ideal):
                valid_ideals_var.append(ideal)
        if valid_ideals_var == []:
            return []
        else:
            valid_ideals.append(valid_ideals_var)
    return [id1+id2+id3+id4 for id1 in valid_ideals[0] for id2 in valid_ideals[1] for id3 in valid_ideals[2] for id4 in valid_ideals[3]] 

def is_ideal_valid(cubic, ideal):
    if cubic.sing_locus.value() in ideal:
        return False
    for poly in list(set([pl.conditions for pl in cubic.tritangent_planes if pl.conditions != 0])):
        if poly in ideal:
            return False
    return True

In [15]:
%time ideals = find_conditions_for_subfamilies(SE1, possible_projs)

0
1
2
3
4
5
6
7
8
9
10
11
CPU times: user 2min 55s, sys: 524 ms, total: 2min 55s
Wall time: 2min 57s


In [16]:
e_sost = {e:c*((1 - 2*i)*c + f)/((1 + 2*i)*c + f)}
d_sost = {d:((1 + 2*i)*c^2 - (2 - 2*i)*c*f + f^2)*(c + f)/(((1 + 2*i)*c + f)*((-1 + 2*i)*c + f))}

In [17]:
cubics = [SE1.reduce(ideal) for ideal in ideals]

In [26]:
[cubic.eqn.variables() for cubic in cubics]

[(x, y, z, t),
 (x, y, z, t),
 (x, y, z, t, d, e, f),
 (x, y, z, t, e, f),
 (x, y, z, t, e, f),
 (x, y, z, t, c, d, e, f),
 (x, y, z, t, e, f),
 (x, y, z, t, d, e, f),
 (x, y, z, t, e, f),
 (x, y, z, t, c, d, e, f),
 (x, y, z, t, c, d, e, f),
 (x, y, z, t, d, e, f),
 (x, y, z, t, c, d, e, f),
 (x, y, z, t, d, e, f),
 (x, y, z, t, e, f),
 (x, y, z, t, e, f),
 (x, y, z, t, c, d, e, f),
 (x, y, z, t, c, d, e, f),
 (x, y, z, t, d, e, f),
 (x, y, z, t, c, d, e, f),
 (x, y, z, t, e, f),
 (x, y, z, t, d, e, f),
 (x, y, z, t, e, f),
 (x, y, z, t, d, e, f),
 (x, y, z, t, e, f),
 (x, y, z, t, e, f),
 (x, y, z, t, e, f),
 (x, y, z, t, e, f),
 (x, y, z, t, d, e, f),
 (x, y, z, t),
 (x, y, z, t),
 (x, y, z, t),
 (x, y, z, t),
 (x, y, z, t, c, d, e, f),
 (x, y, z, t, c, d, e, f),
 (x, y, z, t, c, d, e, f),
 (x, y, z, t),
 (x, y, z, t),
 (x, y, z, t),
 (x, y, z, t),
 (x, y, z, t),
 (x, y, z, t),
 (x, y, z, t, e, f),
 (x, y, z, t, e, f),
 (x, y, z, t, d, e, f),
 (x, y, z, t, c, d, e, f),
 (x, y, z, t)